<a href="https://colab.research.google.com/github/ParhamPishro/Solutions-of-FUM-Academic-Exercises/blob/main/Machine%20Learning/2_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pumpkin Seeds Dataset
in this exercise we have a dataset of 2500 datapoints about pumpkin seeds with two classes. our goal in this exercise is to  create an SVM classifier model that can successfully predict the class to which each pumpkin belongs to. we also want to find the optimal set of hyperparameters for this model. to do this, we do the following steps:

1. Normalizing the data by limiting it's range to (0, 1) and doing train-test split.
2. Running the hyperparameter optimization step on our data.
3. After finding the optimal set of hyperparameters, create an SVM model.
4. after finding the optimal model, analyze the results.

let's begin by importing the important libraries we will be using throughout this exercise:

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import optuna
import warnings
warnings.filterwarnings("ignore")

### 1. Normalize the data and do train-test split.
in this step we import the data, normalize it's features, and do the train-test split. but first we need to import the data and get an idea about it's shape, potential missing datapoints, and the datatypes for its feature columns. We use the pandas library to load the data, and the sklearn library to normalize and train-test split the data.

In [ ]:
df = pd.read_excel("Pumpkin_Seeds_Dataset.xlsx")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Area               2500 non-null   int64  
 1   Perimeter          2500 non-null   float64
 2   Major_Axis_Length  2500 non-null   float64
 3   Minor_Axis_Length  2500 non-null   float64
 4   Convex_Area        2500 non-null   int64  
 5   Equiv_Diameter     2500 non-null   float64
 6   Eccentricity       2500 non-null   float64
 7   Solidity           2500 non-null   float64
 8   Extent             2500 non-null   float64
 9   Roundness          2500 non-null   float64
 10  Aspect_Ration      2500 non-null   float64
 11  Compactness        2500 non-null   float64
 12  Class              2500 non-null   int64  
dtypes: float64(10), int64(3)
memory usage: 254.0 KB


as you can see we have 12 features and 1 label. we don't have any missing values and all the features are numeric. in the next step we seperate the feature columns from our label column, normalize the features, and run train-test split on our data using the sklearn library. it will also be useful to check for any class imbalances in order to choose the right metric to use for measuring our model's performance.

In [ ]:
# seperating the feature data from the label column
features = df.drop(["Class"], axis=1)

# normalizing our features
scaler = MinMaxScaler()
features = pd.DataFrame(scaler.fit_transform(features), columns=features.columns)

# getting the label column
labels = df["Class"]

# checking for class imbalances
count = labels.value_counts()
percentages = count/len(labels) * 100
print("Class percentages:", str(percentages))

# doing the train-test split on the data.
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.7)

Class percentages: Class
0    52.0
1    48.0
Name: count, dtype: float64


as you can see above, 52% of our data belongs to class 1. this means that we have no class imbalance and the accuracy metric is a very appropriate metric to use in this context. so we will be using the accuracy metric for measuring our class's performance.

# 2. Running the hyperparameter optimization step on our data
To do this, we make use of the "optuna" library. a hyperparameter optimization framework that allows us to search the hyperparameter space efficiently! i won't go into the details, but it uses a bayesian approach to make much more educated guesses based on its previous guesses in order to find the optimal hyperparameters faster. this method seems to be generally faster than grid search and random search, especially when we have a lot of hyperparameters to tune at the same time.

to do the optimization, we first define an "objective" function which defines hyperparameters, trains the model using those hyperparameters, and returns a performance metric that we want to minimize. since we wanted to maximize our performance metrics, we can just pass the negative amount of our performance metrics and let optuna do it's magic. but before defining an objective function, let's define a my_SVM function to make writing our objective function more convinient. this function takes featurs, labels, class imbalance status, and the SVM hyperparameters as inputs. and it returns a fitted model, alongside the cross validation performance of our model.

note that the cross validation performance does not reflect the performance of our final model, but rather the performance of the model given our set of hyperparameters. this cross validation score is meant to measure how good our choice of hyperparameters are. the reason we use this instead of the test data for measuring the goodness of our hyperparameters is that we don't want to tune the hyperparameters to optimize for a specific choice of test data.


In [ ]:
def run_SVM(features, target, is_imbalanced = False, C=1, kernel='linear', degree=3, gamma=1, coef0=0, max_iter=100):
    model = svm.SVC(C=C, kernel=kernel, degree=degree, gamma=gamma, coef0=coef0, max_iter=max_iter)
    model.fit(features, target)
    if is_imbalanced:
        score = cross_val_score(model, features, target, scoring='f1', cv=5)
    else: score = cross_val_score(model, features, target, scoring='accuracy', cv=5)
    return model, score.mean()

we have the max_iter hyperparameter which tells the model how many training iterations it can run at max before terminating the training process. this change is made because with some hyperparameters, fitting the model can take and exceptionally long time, for example once it took 8 hours before i terminated the training process early! I will be treating this max_iter as a hyperparameter with an upperbound to make sure the .fit method doesn't run indefinitly.

now that we have the SVM function, let's build our objective function.

In [ ]:
def objective(trial, features, target, is_imbalanced = False):
    kernel = trial.suggest_categorical("kernel", ["linear", 'rbf', 'sigmoid', 'poly'])
    C = trial.suggest_float("C", 1e-3, 1e3, log=True)
    max_iter = trial.suggest_int("max iter", 1e1, 1e6, log=True)
    gamma = 1
    coef0 = 0
    degree = 3
    if kernel != "linear":
        gamma = trial.suggest_float("gamma", 1e-5, 1e3, log=True)
        if kernel == "poly" or kernel == "sigmoid":
            coef0 = trial.suggest_float("coef0", 1e-3, 1e3, log=True)
            if kernel == "poly":
                degree = trial.suggest_int("degree", 2, 6)
    print("kernel: {}, C: {}, gamma: {}, coef0: {}, degree: {}, tol: {}".format(kernel, C, gamma, coef0, degree, max_iter))
    # sometimes the polynomial kernel becomes too large and becomes NaN. this part, handles that error.
    try:
        _, score = run_SVM(features, target, is_imbalanced, C, kernel, degree, gamma, coef0, max_iter)
        return -score
    except ValueError as e:
        return 0

the objective function always takes an input named "trial". a trial is an object that is passed to the objective function during optimization process and it can be used to give us our hyperparameters. the trial object can suggest hyperparameter values which we can use in our training process. here are the ones that has been used here:

kernel = trial.suggest_categorical("kernel", ["linear", 'rbf', 'sigmoid', 'poly']) the suggest_categorical method takes the name of the hyperparameter as it's first input. the second input is a list of objects from which our hyperparameter is chosen. it then chooses one of these values and returns it as an output.

C = trial.suggest_float("C", 1e-3, 1e3, log=True) the sugget_float method in the trial object takes the name of the hyperparameter as it's first argument. its second and third arguments define the lower and upper bound of our search space. the log=True argument states that we care more about the magnitude of the value rather than the exact value itself. it is usually used when we a large search space in terms of logarithmic scale. this method takes a float number from our search space and returns it as output.

max_iter = trial.suggest_int("max iter", 1e1, 1e6, log=True) the suggest_int method works exactly the same way as suggest_float, except that it selects and returns an integer value.

now that we have defined our objective function, it's time to optimize our hyperparameters. to do this, we first create an object called "study". this objects keeps track of our trials, guides the search, and optimizes our hyperparameters. after creating this object, we call the .optimize method on it. the first argument is our objective function. and it's second argument is the number of trials to run. the more trials we try, the better hyperparameters we get, the longer compute time becomes. i tried this optimization with 1000 trials for this model, but you can make it less (100 or less) to see a good demonstration of optuna's functionality.


In [ ]:
study = optuna.create_study()
study.optimize(lambda trial: objective(trial, features, labels), 1000)
best_trial = study.best_trial

print("Best parameters for SVM:")
for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))

[I 2024-11-15 22:50:57,638] A new study created in memory with name: no-name-da466179-2bc2-40ae-a262-b8fb6ef0a383


kernel: linear, C: 55.883507112546084, gamma: 1, coef0: 0, degree: 3, tol: 553601


[I 2024-11-15 22:50:58,060] Trial 0 finished with value: -0.8800000000000001 and parameters: {'kernel': 'linear', 'C': 55.883507112546084, 'max iter': 553601}. Best is trial 0 with value: -0.8800000000000001.
[I 2024-11-15 22:50:58,096] Trial 1 finished with value: -0.6444 and parameters: {'kernel': 'linear', 'C': 0.019249731699725194, 'max iter': 16}. Best is trial 0 with value: -0.8800000000000001.


kernel: linear, C: 0.019249731699725194, gamma: 1, coef0: 0, degree: 3, tol: 16
kernel: sigmoid, C: 0.2287386435626245, gamma: 18.127488408347773, coef0: 4.283117938809895, degree: 3, tol: 316


[I 2024-11-15 22:50:58,347] Trial 2 finished with value: -0.5204 and parameters: {'kernel': 'sigmoid', 'C': 0.2287386435626245, 'max iter': 316, 'gamma': 18.127488408347773, 'coef0': 4.283117938809895}. Best is trial 0 with value: -0.8800000000000001.
[I 2024-11-15 22:50:58,439] Trial 3 finished with value: -0.4856000000000001 and parameters: {'kernel': 'poly', 'C': 0.019394585490750783, 'max iter': 166, 'gamma': 377.0454743668027, 'coef0': 406.42054547840763, 'degree': 6}. Best is trial 0 with value: -0.8800000000000001.


...

kernel: rbf, C: 461.9792311882036, gamma: 2.526445912232787, coef0: 0, degree: 3, tol: 806133


[I 2024-11-15 23:29:33,728] Trial 998 finished with value: -0.8912000000000001 and parameters: {'kernel': 'rbf', 'C': 461.9792311882036, 'max iter': 806133, 'gamma': 2.526445912232787}. Best is trial 182 with value: -0.8936.


kernel: rbf, C: 803.9226215875813, gamma: 1.7397326075854367, coef0: 0, degree: 3, tol: 356334


[I 2024-11-15 23:29:35,742] Trial 999 finished with value: -0.892 and parameters: {'kernel': 'rbf', 'C': 803.9226215875813, 'max iter': 356334, 'gamma': 1.7397326075854367}. Best is trial 182 with value: -0.8936.


Best parameters for SVM:
kernel: rbf
C: 839.9643342771068
max iter: 424468
gamma: 1.9014306765412465


# 3. Creating the SVM model.
in this step we will create the SVM model using our found hyperparameters. since we have already implemented the run_SVM function and have found our optimal hyperparameters, this step will be fairly straght forward.

In [ ]:
model, _ = run_SVM(x_train, y_train, is_imbalanced = False, kernel="rbf", C = 839.964, max_iter=424468, gamma = 1.9)

# 4. Analyze the results
in this step we will measure the accuracy of our model on the test data and and also compare it against a model with default hyperparameter values.

In [ ]:
# creating the model with default hyperparameter values
model_default, _ = run_SVM(x_train, y_train)

y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("optimal model test data accuracy: {}".format(accuracy))

y_pred_default = model_default.predict(x_test)
accuracy_default = accuracy_score(y_test, y_pred_default)
print("default model test data accuracy: {}".format(accuracy_default))

optimal model test data accuracy: 0.8822857142857143
default model test data accuracy: 0.6451428571428571


as you can see, the hyperparameter step was very effective at increasing the performance of our model.